The data shows the information of American colleges and universities in 1995, the detail as below. And we would like to classify good colleges and universities. <br>
[ACollege1995P.csv] Columns:<br>
app - No. of applicants received<br>
acc - No. of applicants accepted<br>
enrol - No. of new students enrolled<br>
ftime - No. of full-time undergraduates<br>
ptime - No. of part-time undergraduate<br>
instate - In-state tuition<br>
outstate - Out-state tuition<br>
rbcost - Room and board cost<br>
bkcost - Estimated book cost<br>
phd - % of faculty with Ph.D.<br>
sfratio - Student/faculty ratio<br>
expend - Instructional expenditure per student<br>
grad - Graduation rate<br>
good - With 30%+ new students from top 10% high school class.

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('../ACollege1995P.csv')
data['normal'] = 1
data.loc[data['good']==1,'normal'] = 0

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data.ix[:,:'grad'].as_matrix()
                            , data.ix[:,'good':'normal'].as_matrix(), test_size=0.3, random_state=852)

In [4]:
learning_rate = 0.001
training_epochs = 10000
batch_size = 300
display_step = 1000

n_hidden_1 = 10
n_hidden_2 = 5
n_input = 13
n_classes = 2

In [5]:
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [6]:
def multilayer_perceptron(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [7]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = multilayer_perceptron(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [8]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(X_train.shape[0]/batch_size)

        for i in range(total_batch):
            batch_x, batch_y = next_batch(batch_size, X_train, y_train)

            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})

            avg_cost += c / total_batch

        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: y_test}))

Epoch: 0001 cost= 20034.439453125
Epoch: 1001 cost= 93.843597412
Epoch: 2001 cost= 1.818534732
Epoch: 3001 cost= 2.458051920
Epoch: 4001 cost= 0.468692124
Epoch: 5001 cost= 0.552739382
Epoch: 6001 cost= 0.425023258
Epoch: 7001 cost= 0.537848115
Epoch: 8001 cost= 0.435653895
Epoch: 9001 cost= 0.435970724
Optimization Finished!
Accuracy: 0.827869


You may get the bad model as random initial starting point is important in neural network.

# Improvement
In general, the standardize transformation can improve the result.

In [9]:
n_input = 7

r_model = data.columns[[3,4,7,8,9,11,12]]
scaler = preprocessing.StandardScaler().fit(data[r_model])
sdata = pd.DataFrame(scaler.transform(data[r_model]), columns=r_model)

X_train, X_test, y_train, y_test = train_test_split(sdata.as_matrix()
                            , data.ix[:,'good':'normal'].as_matrix(), test_size=0.3, random_state=852)

In [10]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = multilayer_perceptron(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [11]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(X_train.shape[0]/batch_size)

        for i in range(total_batch):
            batch_x, batch_y = next_batch(batch_size, X_train, y_train)

            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})

            avg_cost += c / total_batch

        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: y_test}))

Epoch: 0001 cost= 2.457930565
Epoch: 1001 cost= 0.360575467
Epoch: 2001 cost= 0.303899318
Epoch: 3001 cost= 0.307318866
Epoch: 4001 cost= 0.283800423
Epoch: 5001 cost= 0.263422042
Epoch: 6001 cost= 0.250074059
Epoch: 7001 cost= 0.217214659
Epoch: 8001 cost= 0.229974717
Epoch: 9001 cost= 0.254985541
Optimization Finished!
Accuracy: 0.840164
